In [1]:
import numpy as np

pi = np.pi
np.random.seed(123)

In [2]:
signal = np.array([1, 1, 1, 1, 1, 2, 2, 1, 2, 2])
signal = signal.reshape((-1, 1))

In [3]:
Theta = np.linspace(1, 4, 3, endpoint = False).reshape(-1, 1)

In [4]:
def d2(theta, psi):
    diff = np.abs(psi - theta)
    return np.sum(np.square(np.minimum(diff, 2*pi - diff)))

In [5]:
def apart(y, Theta, lda):
    T = y.shape[0]
    M = Theta.shape[0]

    V = np.zeros((T + 1, M))
    s = -1 * np.ones((T + 1, M), dtype=np.int32)
    for t in range(1, T + 1):
        for k in range(M):
            V_candidates = V[t-1] + lda * np.any(Theta[k] != Theta, axis=1) + d2(Theta[k], y[t-1])
            best_idx = np.argmin(V_candidates)
            V[t][k] = V_candidates[best_idx]
            s[t][k] = best_idx

    # Backtracking
    states = np.zeros(T, dtype=np.int32)
    state = np.argmin(V[T])
    for t in reversed(range(T)):
        states[t] = state
        state = s[t + 1][state]
    
    chpnts = np.arange(len(y) - 1)[states[:-1] != states[1:]]
    return chpnts, Theta[states], V, s

penalty = 0.1
chpnts, signal_mean, V, s = apart(signal, Theta, penalty)
print(V)
print(s)
print(chpnts)

[[0.  0.  0. ]
 [0.  1.  4. ]
 [0.  1.1 4.1]
 [0.  1.1 4.1]
 [0.  1.1 4.1]
 [0.  1.1 4.1]
 [1.  0.1 1.1]
 [1.2 0.1 1.2]
 [0.2 1.1 4.2]
 [1.2 0.3 1.3]
 [1.4 0.3 1.4]]
[[-1 -1 -1]
 [ 0  1  2]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 1  1  1]
 [ 1  1  1]
 [ 0  0  0]
 [ 1  1  1]]
[4 6 7]


In [ ]:
def apart_2(y, Theta, lda):
    T = y.shape[0]
    M = Theta.shape[0]

    V = np.zeros((T + 1, M))                        # best cost upto time t and end with level k
    tau = -1 * np.ones((T + 1, M), dtype=np.int32)  # best last time t to change if end with level k
    last_change = -1 * np.ones((T), dtype=np.int32) # best last change location upto time t

    best_prev = 0
    for t in range(1, T + 1):
        for k in range(M):
            if best_prev + lda < V[t - 1][k]:
                V[t][k]   = best_prev + lda
                tau[t][k] = t - 2
            else:
                V[t][k]   = V[t - 1][k]
                tau[t][k] = tau[t - 1][k]
            
            V[t][k] = V[t][k] + d2(Theta[k], y[t-1])

        best_idx = np.argmin(V[t])
        best_prev = V[t][best_idx]
        last_change[t-1] = tau[t][best_idx]
    
    # trace back
    s = last_change[-1]
    chpnts = np.array([], dtype=int)
    while s > 0:
        chpnts = np.append(s, chpnts)
        s = last_change[s]
    
    # get mean
    ext_chpnts = np.append(np.append(-1, chpnts), len(signal)-1)
    signal_mean = np.zeros_like(signal)
    for i in range(len(ext_chpnts) - 1):
        start = ext_chpnts[i]
        end = ext_chpnts[i + 1]
        signal_mean[start + 1: end + 1] = Theta[np.argmin(V[end + 1])]

    return V, tau, last_change, chpnts, signal_mean

penalty = 0.1
V, tau, last_change, chpnts = apart_2(signal, Theta, penalty)
print(V)
print(tau)
print(last_change)
print(chpnts)

[[0.  0.  0. ]
 [0.  1.  4. ]
 [0.  1.1 4.1]
 [0.  1.1 4.1]
 [0.  1.1 4.1]
 [0.  1.1 4.1]
 [1.  0.1 1.1]
 [1.2 0.1 1.2]
 [0.2 1.1 4.2]
 [1.2 0.3 1.3]
 [1.4 0.3 1.4]]
[[-1 -1 -1]
 [-1 -1 -1]
 [-1  0  0]
 [-1  1  1]
 [-1  2  2]
 [-1  3  3]
 [-1  4  4]
 [ 5  4  5]
 [ 6  4  6]
 [ 6  7  7]
 [ 8  7  8]]
[-1 -1 -1 -1 -1  4  4  6  7  7]
[4 6 7]


In [24]:
ext_chpnts = np.append(np.append(-1, chpnts), len(signal)-1)
mean = np.zeros_like(signal)
for i in range(len(ext_chpnts) - 1):
    start = ext_chpnts[i]
    end = ext_chpnts[i + 1]
    mean[start + 1: end + 1] = Theta[np.argmin(V[end + 1])]

In [25]:
mean

array([[1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [1],
       [2],
       [2]])